# Modis-tempCNN mapping model
Train and save the Modis-tempCNN model for map generation 

In [ ]:
import os
import json

import initialise
import common
from modelling_functions import create_models
from architecture_modis_tempCNN import model_params
from model_parameters import ModelParams, ExperimentParams
import scenarios

## Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `aux_csv`: The file containing extracted DEM and other auxiliary data for each sample, created by `Extract DEM Data.ipynb`. This includes the labels and pre-processed site location and sampling date data from Globe-LFMC

In [ ]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up model parameters

### Model parameters settings
To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [ ]:
# Customize model parameters
scenarios.common_params(model_params)
model_params['modelName'] = 'modis-tempCNN_map2017'
model_params['description'] = 'Build model for 2017 maps'
model_params['samplesFile'] = aux_csv
model_params['saveModels'] = [common.MODIS_TEMPCNN_MODEL]   # Save the model to use for map-wide estimates
model_params['splitMethod'] = None                          # Train using all the data
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])

model_params.add_input('optical', {'filename': modis_csv, 'channels': 7})

model_params

## Build and run the model
Builds and trains the LFMC model. After training the model, several derived models are created and evaluated. The full list of derived models is:
- `base` - The fully trained model
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.

The model and training statistics are saved to `model_dir`.

In [ ]:
model = create_models(model_params)